In [1]:
import pandas as pd
import numpy as np

In [2]:
vistas = pd.read_csv("procesado1/VistasCCorrecta.csv")
postulaciones = pd.read_csv("Predecir/df_postulaciones.csv")

In [3]:
vistas = vistas.rename(columns = {"idAviso":"idaviso"})
vistas = pd.concat([vistas,postulaciones])
vistas = vistas.drop_duplicates(["idaviso","idpostulante"])
postulaciones = postulaciones.sample(frac=0.001)

avisos = pd.read_csv("Predecir/avisos_detalles_c.csv")
postulantes = pd.read_csv("Predecir/postulantes_datos_c.csv")

In [4]:
vistas["nivel_laboral"]= pd.merge(vistas,avisos.loc[:,["idaviso","nivel_laboral"]], \
                                  on="idaviso",how="inner")["nivel_laboral"]

In [5]:
vistas = vistas.drop_duplicates(["idaviso","idpostulante"])

In [6]:
vistas = vistas.reset_index().drop(columns = "index")

In [7]:
del vistas["Unnamed: 0"]
del vistas["Unnamed: 0.1"]
del postulantes["Unnamed: 0"]

In [8]:
names = ["CantidadVistasXNivelLaboral0","CantidadVistasXNivelLaboral1","CantidadVistasXNivelLaboral2","CantidadVistasXNivelLaboral3","CantidadVistasXNivelLaboral4","CantidadVistasXNivelLaboral5"]
for i in range(0,6):
    postulantes = pd.merge(vistas.groupby(["idpostulante","nivel_laboral"]).count().reset_index() \
        .rename(columns = {"idaviso":names[i]}).loc[vistas["nivel_laboral"]==i] \
        .loc[:,["idpostulante",names[i]] ] \
        ,postulantes,on="idpostulante",how="right")


In [9]:
#solo el predictor
for i in range(0,6):
    postulantes[names[i]] = postulantes[names[i]].fillna(6)

In [10]:
postulantes = postulantes.drop_duplicates("idpostulante")

In [11]:
vistas = vistas.rename(columns = {"nivel_laboral":"empresa"})
vistas["empresa"]= pd.merge(vistas,avisos.loc[:,["idaviso","denominacion_empresa"]], \
                                  on="idaviso",how="inner")["denominacion_empresa"]

In [12]:
postulantes = pd.merge(vistas.groupby(["idpostulante","empresa"]).count().reset_index() \
        .rename(columns = {"idaviso":"EmpresaMasVista"}) \
        .loc[:,["idpostulante","EmpresaMasVista" ]].groupby("idpostulante").max().reset_index() 
        ,postulantes,on="idpostulante",how="right")

In [13]:

postulantes["EmpresaMasVista"] = postulantes["EmpresaMasVista"].fillna(10000)

In [14]:
vistas = vistas.rename(columns = {"empresa":"nombre_area"})
vistas["nombre_area"]= pd.merge(vistas.drop(columns="nombre_area"),avisos.loc[:,["idaviso","nombre_area"]], \
                                  on="idaviso",how="inner")["nombre_area"]

In [15]:
postulantes = pd.merge(vistas.groupby(["idpostulante","nombre_area"]).count().reset_index() \
        .rename(columns = {"idaviso":"AreaMasVista"}) \
        .loc[:,["idpostulante","AreaMasVista" ]].groupby("idpostulante").max().reset_index() 
        ,postulantes,on="idpostulante",how="right")

In [16]:
postulantes["AreaMasVista"] = postulantes["AreaMasVista"].fillna(10000)

In [19]:
postulantes.isna().values.any()

False

In [18]:
postulantes.to_csv("Predecir/postulantes_con_vistas.csv")